In [ ]:
%pip install langdetect

In [1]:
import re
import sys
import string
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer 

from langdetect import detect

from parapply import parapply

from sklearn.metrics import accuracy_score, classification_report

import fasttext

import warnings
warnings.filterwarnings('ignore')

data = pd.read_csv(sys.path[0]+'/train.csv',sep=',',dtype=object)
data.shape

(497819, 4)

In [10]:
def preprocessing(data,n_jobs = 1):

    for character in string.punctuation:
        data['name_1'] = data['name_1'].apply(lambda x: str(x).replace(character, ''))
        data['name_2'] = data['name_2'].apply(lambda x: str(x).replace(character, ''))

    data['lang_1'] = parapply(data['name_1'],lambda x: detect(x),n_jobs=n_jobs)
    data['lang_2'] = parapply(data['name_2'],lambda x: detect(x),n_jobs=n_jobs)

    lang_ru_indexes =data[data['lang_1']=='ru'].index
    print(lang_ru_indexes)
    if len(list(lang_ru_indexes))==0:
        lang_other_indexes = data.index
    else:
        lang_other_indexes =data.index.drop(lang_ru_indexes)
    stemmer_ru = SnowballStemmer("russian")
    stemmer_other = SnowballStemmer("english")

    if len(list(lang_ru_indexes))!=0:
        data.loc[lang_ru_indexes,'name_1_stemmed'] = parapply(data.loc[lang_ru_indexes,'name_1'],lambda x: stemmer_ru.stem(x),n_jobs=4)
        data.loc[lang_other_indexes,'name_1_stemmed'] = parapply(data.loc[lang_other_indexes,'name_1'],lambda x: stemmer_other.stem(x),n_jobs=4)
    else:
        data.loc[lang_other_indexes,'name_1_stemmed'] = parapply(data.loc[lang_other_indexes,'name_1'],lambda x: stemmer_other.stem(x),n_jobs=4)

    lang_ru_indexes =data[data['lang_2']=='ru'].index
    print(lang_ru_indexes)
    if len(list(lang_ru_indexes))==0:
        lang_other_indexes = data.index
    else:
        lang_other_indexes =data.index.drop(lang_ru_indexes)
    stemmer_ru = SnowballStemmer("russian")
    stemmer_other = SnowballStemmer("english")

    if len(list(lang_ru_indexes))!=0:
        data.loc[lang_ru_indexes,'name_2_stemmed'] = parapply(data.loc[lang_ru_indexes,'name_2'],lambda x: stemmer_ru.stem(x),n_jobs=n_jobs)
        data.loc[lang_other_indexes,'name_2_stemmed'] = parapply(data.loc[lang_other_indexes,'name_2'],lambda x: stemmer_other.stem(x),n_jobs=n_jobs)
    else:
        data.loc[lang_other_indexes,'name_2_stemmed'] = parapply(data.loc[lang_other_indexes,'name_2'],lambda x: stemmer_other.stem(x),n_jobs=n_jobs)
   
    
    return data


def cos_sim(a, b):
	dot_product = np.dot(a, b)
	norm_a = np.linalg.norm(a)
	norm_b = np.linalg.norm(b)
	return dot_product / (norm_a * norm_b)

In [4]:
# dataset = preprocessing(data,4)

Int64Index([  3371,   4944,   5885,  19271,  19550,  23412,  23865,  24518,
             26757,  28619,
            ...
            482326, 483872, 488071, 488150, 489129, 489726, 493907, 493995,
            494227, 497367],
           dtype='int64', length=176)
Int64Index([   384,    469,   3808,   4354,   5153,   5450,   5470,   5817,
              6114,   7185,
            ...
            489237, 489511, 489553, 491734, 492110, 494314, 495064, 495335,
            497139, 497231],
           dtype='int64', length=833)


In [5]:
# dataset.to_csv(sys.path[0]+'/CompanyNamesSimularity_dataset.csv',sep=';',index=False)

In [2]:
dataset = pd.read_csv(sys.path[0]+'/CompanyNamesSimularity/CompanyNamesSimularity_dataset.csv',sep=';',dtype=object)

In [4]:
model_1 = fasttext.train_unsupervised(input=sys.path[0]+'/train_strings.csv',model="skipgram", minn=2, maxn=5, dim=300,epoch=5, lr=0.1,thread=10)
model_2 = fasttext.train_unsupervised(input=sys.path[0]+'/train_strings.csv',model="skipgram", minn=2, maxn=5, dim=150,epoch=7, lr=0.05,thread=10)
model_3 = fasttext.train_unsupervised(input=sys.path[0]+'/train_strings.csv',model="skipgram", minn=2, maxn=5, dim=100,epoch=10, lr=0.01,thread=10)

Read 4M words
Number of words:  16376
Number of labels: 0
Progress: 100.0% words/sec/thread:   91671 lr:  0.000000 avg.loss:  0.560611 ETA:   0h 0m 0s 11.4% words/sec/thread:   92038 lr:  0.088605 avg.loss:  1.401040 ETA:   0h 0m22s 79.4% words/sec/thread:   91801 lr:  0.020582 avg.loss:  0.620099 ETA:   0h 0m 5s
Read 4M words
Number of words:  16376
Number of labels: 0
Progress: 100.0% words/sec/thread:  148018 lr:  0.000000 avg.loss:  0.524437 ETA:   0h 0m 0s 22.9% words/sec/thread:  150333 lr:  0.038560 avg.loss:  1.017529 ETA:   0h 0m16s 29.7% words/sec/thread:  152404 lr:  0.035157 avg.loss:  0.888536 ETA:   0h 0m15s 39.3% words/sec/thread:  151888 lr:  0.030355 avg.loss:  0.785320 ETA:   0h 0m13s 64.6% words/sec/thread:  150659 lr:  0.017675 avg.loss:  0.631490 ETA:   0h 0m 7s 81.4% words/sec/thread:  149454 lr:  0.009293 avg.loss:  0.569469 ETA:   0h 0m 4s
Read 4M words
Number of words:  16376
Number of labels: 0
Progress: 100.0% words/sec/thread:  189289 lr:  0.000000 avg.loss:

In [5]:
model_1.save_model(sys.path[0]+'/model_1.bin')
model_2.save_model(sys.path[0]+'/model_2.bin')
model_3.save_model(sys.path[0]+'/model_3.bin')


In [ ]:
model_1 = fasttext.load_model(sys.path[0]+'/model_1.bin')

In [6]:
dataset['is_duplicate'] = dataset['is_duplicate'].astype(int)

In [7]:
dataset_val = dataset[dataset['is_duplicate']==1]
dataset_val = dataset_val.reset_index(drop=True)
Y = dataset['name_2_stemmed']
print(dataset_val.shape, Y.shape)


(3658, 8) (497819,)


In [8]:
embeddings_Y = np.zeros((dataset.shape[0],300),dtype='float32')
for i,sentence in enumerate(Y):
    embeddings_Y[i,:] = model_1.get_sentence_vector(sentence)

In [25]:
dataset_val['names_1_simularity'] = None
dataset_val['min_1_distance'] = 0
results_tuple_list = []

def simularity(y,Y=Y,embeddings = embeddings_Y,model=model_1):
    distances_argmin = []
    simularity_values = []
    distances = np.zeros(embeddings.shape[0])
    vector_i = model.get_sentence_vector(y) 

    for j, sentence_j in enumerate(Y):
        vector_j = embeddings[j,:]
        distances[j] = cos_sim(vector_i,vector_j)

    distances_argmin = Y[np.argmax(distances)]
    simularity_values = np.max(distances)

    return (distances_argmin, simularity_values)

results_tuple_list.append(dataset_val['name_1_stemmed'].apply(lambda x: simularity(x)))



KeyboardInterrupt: 

In [ ]:
dataset_val['names_1_simularity'] = [i[0] for i in results_tuple_list]
dataset_val['target'] = dataset_val['name_2_stemmed']==dataset_val['names_1_simularity']
dataset_val['target'] = dataset_val['target'].astype(int)
dataset_val['is_duplicate'] = dataset_val['is_duplicate'].astype(int)


print('accuracy_score= ',round(accuracy_score(data_1['is_duplicate'],data_1['target']),2))